In [ ]:
# Imports
import pandas as pd
import numpy as np
import urllib.request
import xml.etree.ElementTree as et 

print("Done importing libraries!")

In [ ]:
# ENTER list of metabolites by HMDB ID

metab_list = ['HMDB0030396', 'HMDB0000122', 'HMDB0000001', 'HMDB0000134']

In [ ]:
# ENTER data/columns to store 
# For example use http://www.hmdb.ca/metabolites/HMDB0000149.xml and find correct column names

columns = ['accession',
           'name',
           'synonyms/synonym',
           'chemical_formula',
           'monisotopic_molecular_weight',
           'cas_registry_number',
           'kegg_id',
           'pubchem_compound_id',
           'chemspider_id',
           'smiles',
           'inchi',
           'taxonomy/super_class',
           'taxonomy/class',
           'taxonomy/sub_class',
            'biological_properties/pathways/pathway/name']


In [ ]:
# RUN cell (don't need to edit this)
# Store all information from HMDB into hmdb_table

hmdb_table = pd.DataFrame(columns=columns)  # Empty table

for metabolite in metab_list:

    # ensure HMDB ID is new  (i.e. 11 characters) 
    if len(metabolite) == 9:
        metabolite = metabolite[:4] + "00" + metabolite[4:] 

    # Request .xlm for the metabolite
    link = 'http://www.hmdb.ca/metabolites/' + metabolite + '.xml'
    try:
        urllib.request.urlretrieve(link, "hmdb_metabolites.xml")
    except:
        raise ValueError("HMDB ID: {} does not exist".format(metabolite))
    
    # Open xlm
    file_xml = open("hmdb_metabolites.xml").read()
    root = et.XML(file_xml)

    # Loop through each column and store value
    values = []
    for col in columns:
         # Extract info as an element
        col_element = et.fromstring(file_xml).findall(col)
        
        # Convert element to string and append to values if it exists
        if len(col_element) == 0:
            col_value = ""
        else:
            col_value = col_element[0].text 
        
        # If there are additional columns add a comma e.g. 'A, B, C'
        if len(col_value) > 1:
            for i in col_element[1:]:
                col_value = col_value + ", " + i.text
        values.append(col_value)
    
    # Create a pandas series, and add it to hmdb_table 
    row = pd.Series(values, index=columns)
    hmdb_table = hmdb_table.append(row, ignore_index=True)
        
print("Done! Check hmdb_table below.")

In [ ]:
# View table

display(hmdb_table)

In [ ]:
# EXPORT as excel
file_excel = "hmdb_table.xlsx"
hmdb_table.to_excel(file_excel)
print("Done.")